In [1]:
from fn import op, _
from itertools import repeat
from functools import reduce
# from objproxies import LazyProxy
import json
from copy import deepcopy, copy
from pipetools import pipe
from functools import partial

In [2]:
state_dict = {
    's1': 2,
    's2': 4,
    's3': 300
}

In [3]:
# def pipeline(*steps):
#     return reduce(lambda x, y: y(x), list(steps))
# def compose(*funcs):
#     return lambda x: reduce(lambda f, g: g(f), list(funcs), x)

In [12]:
# ToDo:
# Handle case where Mechanisms have no input. Perhaps the sentinel value of 0

In [13]:
# UI Behavior Mechanisms per Mechanism
def b1m1(s):
    return s['s1']
def b2m1(s):
    return s['s1'] * s['s2']

def b1m2(s):
    return s['s1']
def b2m2(s):
    return s['s1'] / s['s2']

# UI State Mechanisms per Mechanism
def s1m1(s, _input):
    s['s1'] = s['s1']**2 + _input
def s2m1(s, _input):
    s['s2'] = s['s2'] + 1 + _input  
def s3m1(s, _input):
    s['s3'] = s['s3']

def s1m2(s, _input):
    s['s1'] = s['s1'] + _input
def s2m2(s, _input):
    s['s2'] = s['s2']
def s3m2(s, _input):
    s['s3'] = s['s3'] + 1
    
j = {
    "m1": {
        "behaviors": {
            "b1": b1m1,
            "b2": b2m1
        },
        "states": {
            "s1": s1m1,
            "s2": s2m1,
            "s3": s3m1
        }
    },
    "m2": {
        "behaviors": {
            "b1": b1m2,
            "b2": b2m2
        },
        "states": {
            "s1": s1m2,
            "s2": s2m2,
            "s3": s3m2
        }
    }
}

In [57]:
def generate_configs(j):
    return list(
        map(
            lambda x: (
                list(j[x]["states"].values()),
                list(j[x]["behaviors"].values())
            ), 
            j.keys()
        )
    )

def getColResults(s, funcs):
    return list(map(lambda f: f(s), funcs))

def getBehaviorInput(s, funcs): 
    return op.foldr(_ + _)(getColResults(s, funcs))

def mech_step(sL, state_funcs, behavior_funcs):
    in_copy, out_copy, mutatable_copy = deepcopy(sL), deepcopy(sL), deepcopy(sL)
    last_in_obj, last_mut_obj = in_copy[-1], mutatable_copy[-1]
    
    _input = getBehaviorInput(last_in_obj, behavior_funcs)

    for f in state_funcs:
        f(last_mut_obj, _input)
    
    out_copy.append(last_mut_obj)
    return out_copy

def pipeline(states_list, configs):
    for config in configs:
        s_conf, b_conf = config[0], config[1]
        states_list = mech_step(states_list, s_conf, b_conf)
    return states_list

In [59]:
states_list = [state_dict]
configs = generate_configs(j)
pipeline(states_list, configs)

[{'s1': 2, 's2': 4, 's3': 300},
 {'s1': 14, 's2': 15, 's3': 300},
 {'s1': 28.933333333333334, 's2': 15, 's3': 301}]

In [60]:
states_list

[{'s1': 2, 's2': 4, 's3': 300}]

In [44]:
sm1 = [s1m1, s2m1, s3m1]
bm1 = [b1m1, b2m1]

sm2 = [s1m2, s2m2, s3m2]
bm2 = [b1m2, b2m2]

configs = [(sm1, bm1), (sm2, bm2)]
configs

[([<function __main__.s1m1(s, _input)>,
   <function __main__.s2m1(s, _input)>,
   <function __main__.s3m1(s, _input)>],
  [<function __main__.b1m1(s)>, <function __main__.b2m1(s)>]),
 ([<function __main__.s1m2(s, _input)>,
   <function __main__.s2m2(s, _input)>,
   <function __main__.s3m2(s, _input)>],
  [<function __main__.b1m2(s)>, <function __main__.b2m2(s)>])]

In [139]:
# UI State Mechanisms

# s['s1']^2 + (s['s1'] + (s['s1'] * s['s2']))
# 14 = 4 + (2 + (2 * 4))
def m(sL, s_funcs, b_funcs):
    dSL = deepcopy(sL)
    in_copy = deepcopy(sL)
    out_copy = deepcopy(sL)
    s = dSL[-1]
    _input = getColInput(in_copy[-1], b_funcs)
    
#     s['s1'] = s['s1']**2 + _input
#     s['s2'] = s['s2'] + 1 + _input
    for f in s_funcs:
        f(s, _input)
    
    out_copy.append(s)
    return out_copy

# s['s1'] + (s['s1'] + (s['s1'] / s['s2']))
# 28.9 = 14 + (14 + (14/15))
def m2(sL):
    dSL = deepcopy(sL)
    s = dSL[-1]
    _input = getColInput(deepcopy(sL)[-1], [b1m2, b2m2])
    
    s['s1'] = s['s1'] + _input
    s['s2'] = s['s2']
    
    sL.append(s)
    return sL

# def s2m1(sL):
#     s = deepcopy(sL)[-1]
#     s['s2'] = s['s2'] + 1 + getColInput(s, [b1m2, b2m2])
#     sL.append(s)
#     return sL

# def s2m2(sL):
#     s = deepcopy(sL)[-1]
#     s['s2'] = s['s2']
#     sL.append(s)
#     return sL

In [152]:
pipeline3(states_list, configs)

[{'s1': 2, 's2': 4, 's3': 300},
 {'s1': 14, 's2': 15, 's3': 300},
 {'s1': 28.933333333333334, 's2': 15, 's3': 300}]

In [96]:
def pipeline2(sL, funcs):
    dSL = deepcopy(sL)
    for f in funcs:
        dSL = f(dSL)
    return dSL

pipeline2(states_list, [s1m1, s1m2])

TypeError: s1m1() missing 1 required positional argument: '_input'

In [101]:
state_dict = {
    's1': 2,
    's2': 4,
    's3': 300
}
# l = [state_dict]
# l.append(state_dict)
# l
# print(s1m1([state_dict]))

In [97]:
state_dict
states_list = [state_dict]
states_list

[{'s1': 2, 's2': 4, 's3': 300}]

In [140]:
pipeline(s1m1, s1m2)

TypeError: 'function' object is not subscriptable

In [99]:
pipeline2(states_list, [s1m1, s1m2])

[{'s1': 2, 's2': 4, 's3': 300}]

In [40]:
# def a(x):
#     return x + 1
# func = F() >> s1m1 >> s1m1
# func(state_dict)
def s1_pipe(x): return pipe | s1m1 | s1m2 

In [44]:
s1_pipe(state_dict)

s1m1 | s1m2

In [45]:
state_dict

{'s1': 2, 's2': 4, 's3': 300}

In [125]:
from functools import partial

def multiply(x,y):
        return x * y

# create a new function that multiplies by 2
dbl = partial(multiply,2)
print(dbl(4))

6
